In [1]:
# Import Statements
import eeg_utils
import numpy as np
from eeg_utils import *
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
# Load EEG Data
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [3]:
# Visualize EEG Data Shapes
print("Training/Valid data shape: {}".format(X_train_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Training/Valid target shape: {}".format(y_train_valid.shape))
print("Test target shape: {}".format(y_test.shape))
print("Person train/valid shape: {}".format(person_train_valid.shape))
print("Person test shape: {}".format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
# [RUN IFF] Trained on Subject One ONLY
patient_data = []
patient_truths = []

test_data = []
test_truths = []

for patient in np.unique(person_train_valid):
    idx = np.where(person_train_valid == patient)[0]
    patient_data.append(X_train_valid[idx, :, :])
    patient_truths.append(y_train_valid[idx])
    pass

# ALso split the testing set, as we want to optimize on subject one
for patient in np.unique(person_test):
    idx = np.where(person_test == patient)[0]
    test_data.append(X_test[idx, :, :])
    test_truths.append(y_test[idx])
    pass

# Store actual test data
X_test_actual = X_test
y_test_actual = y_test

X_train_valid = patient_data[0]
y_train_valid = patient_truths[0]
X_test = test_data[0]
y_test = test_truths[0]

In [5]:
# Preprocess Data
data_config = {"Duplicate": 2, "Mu": 0.0, "Sigma": 0.5,
               "Sample": False, "Sample_Step": 5, "Attention": False, "Swap": True}
X_train_valid, y_train_valid, X_test, y_test = preprocess_data(X_train_valid,
                                                               y_train_valid,
                                                               X_test,
                                                               y_test, 
                                                               data_config)

In [6]:
# Split Training/Validation Data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,
                                                      y_train_valid,
                                                      test_size=0.2,
                                                      random_state=0)

In [7]:
# Trim Data pre-Model
X_train_5d = X_train[:, :500, :, :]
X_valid_5d = X_valid[:, :500, :, :]
X_test_5d = X_test[:, :500, :, :]

In [8]:
# Load Optimal Model
model_name = "./CNN_Train_One" # Despite its name, this is still the same CRNN as CRNN_Train_All
optim_model = keras.models.load_model(model_name) # Model that was trained on subject one only

In [9]:
# Evaluate all results (Train on one subject, test on one subject)
optim_model.evaluate(X_train_5d, y_train) # 98.15%
optim_model.evaluate(X_valid_5d, y_valid) # 81.05%
optim_model.evaluate(X_test_5d, y_test) # 70.00%

2/2 [==============================] - 0s 15ms/step - loss: 1.5524 - accuracy: 0.7000


[1.5524128675460815, 0.699999988079071]

In [10]:
# Preprocess all testing data
X_test_actual = X_test_actual.reshape((X_test_actual.shape)[0],
                                     (X_test_actual.shape)[1],
                                     (X_test_actual.shape)[2], 1)
X_test_actual = np.swapaxes(np.swapaxes(X_test_actual, 1, 2), 2, 3)

# Preprocess all testing truths
y_test_actual = y_test_actual - 769
y_test_actual = to_categorical(y_test_actual, 4)

# Separate Data by Patient
patient_data = []
patient_truths = []

for patient in np.unique(person_test):
    idx = np.where(person_test == patient)[0]
    patient_data.append(X_test_actual[idx, :500, :, :])
    patient_truths.append(y_test_actual[idx])
    pass

In [11]:
# Evaluate model on each patient (Train on subject one, Test on a single subject)
for i in range(len(patient_data)):
    optim_model.evaluate(patient_data[i], patient_truths[i])

2/2 [==============================] - 0s 16ms/step - loss: 3.5758 - accuracy: 0.3191


In [12]:
'''
Model Parameters: 122 Epochs
    - Data augmented CRNN with doubly stacked LSTM layer
    - Regularization : 0.006
    - Learning Rate : 0.001
    - Attention : False
    - Time Points : 500
    - Kernel : (10, 1)
    - Sample : False
    - Dropout : 0.5
    - Pool : (3, 1)
    - Duplicate : 2
    - Sigma : 0.5
    - Mu : 0.0
    
    N.B. For importance of the parameters above, see CRNN Model notebook for architecture
'''

'\nModel Parameters: 122 Epochs\n    - Data augmented CRNN with doubly stacked LSTM layer\n    - Regularization : 0.006\n    - Learning Rate : 0.001\n    - Attention : False\n    - Time Points : 500\n    - Kernel : (10, 1)\n    - Sample : False\n    - Dropout : 0.5\n    - Pool : (3, 1)\n    - Duplicate : 2\n    - Sigma : 0.5\n    - Mu : 0.0\n    \n    N.B. For importance of the parameters above, see CRNN Model notebook for architecture\n'